<a href="https://colab.research.google.com/github/dcandrader/IA_2025.2/blob/main/05%20-%20modelo%20eliminando%20programa%20acad%C3%A9mico%20y%20TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Carga de librerias y recursos
import pandas as pd
import os
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from progressbar import progressbar as pbar

In [ ]:
#Carga de datos directamente de Kaggle
os.environ['KAGGLE_CONFIG_DIR'] = "."

In [ ]:
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.66GB/s]


In [ ]:
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip

Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
#Lectura de datos
d_train = pd.read_csv('train.csv')
d_train = d_train.drop(columns=["E_PRGM_ACADEMICO"])

orden_valor = [
    'bajo',
    'medio-bajo',
    'medio-alto',
    'alto'
  ]

d_train['RENDIMIENTO_GLOBAL'] = pd.Categorical(
    d_train['RENDIMIENTO_GLOBAL'],
    categories=orden_valor,
    ordered=True
  ).codes

d_test = pd.read_csv('test.csv')
d_test = d_test.drop(columns=["E_PRGM_ACADEMICO"])
lentr = len(d_train)

d_train

,ID,PERIODO_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,Si,N,No,Si,Si,Postgrado,2,0.322,0.208,0.310,0.267
1,645256,20212,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,No,N,No,Si,No,Técnica o tecnológica incompleta,0,0.311,0.215,0.292,0.264
2,308367,20203,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,No,No,Si,Secundaria (Bachillerato) completa,0,0.297,0.214,0.305,0.264
3,470353,20195,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,No,N,No,Si,Si,Secundaria (Bachillerato) completa,3,0.485,0.172,0.252,0.190
4,989032,20212,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,Si,N,No,Si,Si,Primaria completa,1,0.316,0.232,0.285,0.294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,20195,LA GUAJIRA,Entre 500 mil y menos de 1 millón,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Si,No,N,Si,Si,Si,Secundaria (Bachillerato) incompleta,2,0.237,0.271,0.271,0.311
692496,754213,20212,NORTE SANTANDER,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Primaria incompleta,Si,No,N,No,Si,Si,Secundaria (Bachillerato) incompleta,0,0.314,0.240,0.278,0.260
692497,504185,20183,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Menos de 10 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,No,N,Si,Si,Si,Secundaria (Bachillerato) incompleta,1,0.286,0.240,0.314,0.287
692498,986620,20195,TOLIMA,Entre 2.5 millones y menos de 4 millones,Menos de 10 horas,Estrato 1,No,Primaria completa,No,No,N,Si,Si,No,Primaria completa,0,0.132,0.426,0.261,0.328


In [ ]:
d_train["RENDIMIENTO_GLOBAL"].value_counts()

,count
RENDIMIENTO_GLOBAL,
3,175619
0,172987
1,172275
2,171619


In [ ]:
#Función para preprocesado de datos

def procesado_datos(d):
  for col in d.select_dtypes(include='object').columns:
    d[col] = d[col].astype('category')

  k = d.isna().sum()
  cols_con_faltantes = k[k != 0].index
  for col in cols_con_faltantes:
    if col in d.columns:
        d[col] = d[col].cat.add_categories(['Desconocido'])
        d[col] = d[col].fillna('Desconocido')

  orden = sorted(d['PERIODO_ACADEMICO'].unique())
  mapa_periodos = {valor: i for i, valor in enumerate(orden)}

  d['PERIODO_ACADEMICO'] = d['PERIODO_ACADEMICO'].map(mapa_periodos)
  d['PERIODO_ACADEMICO'] = d['PERIODO_ACADEMICO'].astype('Int64')

  cols_normal = ['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']
  scaler = StandardScaler()
  d_train[cols_normal] = scaler.fit_transform(d_train[cols_normal])

  orden_valor = [
    'Menos de 500 mil',
    'Entre 500 mil y menos de 1 millón',
    'Entre 1 millón y menos de 2.5 millones',
    'Entre 2.5 millones y menos de 4 millones',
    'Entre 4 millones y menos de 5.5 millones',
    'Entre 5.5 millones y menos de 7 millones',
    'Más de 7 millones',
    'No pagó matrícula',
    'Desconocido'
  ]
  d['E_VALORMATRICULAUNIVERSIDAD'] = pd.Categorical(
    d['E_VALORMATRICULAUNIVERSIDAD'],
    categories=orden_valor,
    ordered=True
  ).codes

  orden_horas = [
    '0',
    'Menos de 10 horas',
    'Entre 11 y 20 horas',
    'Entre 21 y 30 horas',
    'Más de 30 horas',
    'Desconocido'
  ]
  d['E_HORASSEMANATRABAJA'] = pd.Categorical(
    d['E_HORASSEMANATRABAJA'],
    categories=orden_horas,
    ordered=True
  ).codes

  orden_estrato = [
    'Estrato 1',
    'Estrato 2',
    'Estrato 3',
    'Estrato 4',
    'Estrato 5',
    'Estrato 6',
    'Sin Estrato',
    'Desconocido'
  ]

  d['F_ESTRATOVIVIENDA'] = pd.Categorical(
    d['F_ESTRATOVIVIENDA'],
    categories=orden_estrato,
    ordered=True
  ).codes

  orden_educacion = [
    'Ninguno',
    'Primaria incompleta',
    'Primaria completa',
    'Secundaria (Bachillerato) incompleta',
    'Secundaria (Bachillerato) completa',
    'Técnica o tecnológica incompleta',
    'Técnica o tecnológica completa',
    'Educación profesional incompleta',
    'Educación profesional completa',
    'Postgrado',
    'No Aplica',
    'No sabe',
    'Desconocido'
  ]

  for col in ['F_EDUCACIONPADRE', 'F_EDUCACIONMADRE']:
    d[col] = pd.Categorical(
        d[col],
        categories=orden_educacion,
        ordered=True
    ).codes

  cat_cols = [c for c in d.select_dtypes(include=['category']).columns if c!='RENDIMIENTO_GLOBAL']

  encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=True)
  encoded = encoder.fit_transform(d[cat_cols])

  encoded_df = pd.DataFrame.sparse.from_spmatrix(
    encoded,
    columns=encoder.get_feature_names_out(cat_cols),
    index=d.index
)

  d_encoded = pd.concat([d.drop(columns=cat_cols), encoded_df], axis=1)

  return d_encoded

In [ ]:
#Preparación para entrenamiento
d_columnas = [i for i in d_train.columns if i!="RENDIMIENTO_GLOBAL"]
all_data = pd.concat((d_train[d_columnas], d_test[d_columnas]))
all_data.index = range(len(all_data))
all_data = procesado_datos(all_data)

In [ ]:
Xtr, ytr = all_data.iloc[:lentr].values, d_train["RENDIMIENTO_GLOBAL"].values
Xts      = all_data.iloc[lentr:].values

print (Xtr.shape, ytr.shape)
print (Xts.shape)

(692500, 62) (692500,)
(296786, 62)


In [ ]:
Xtr = Xtr.astype('float32')
Xts = Xts.astype('float32')
ytr = ytr.astype('int32')

In [ ]:
Xtr.dtype, Xts.dtype, ytr.dtype

(dtype('float32'), dtype('float32'), dtype('int32'))

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input
import numpy as np

n_features = Xtr.shape[1]
n_classes = len(np.unique(ytr))

model = Sequential([
    Input(shape=(n_features,)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(n_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(Xtr, ytr, epochs=30, batch_size=32)

# Predicciones
preds_ts = model.predict(Xts)
preds_classes = preds_ts.argmax(axis=1)

Epoch 1/30
21641/21641 ━━━━━━━━━━━━━━━━━━━━ 52s 2ms/step - accuracy: 0.2535 - loss: 643.4483
Epoch 2/30
21641/21641 ━━━━━━━━━━━━━━━━━━━━ 49s 2ms/step - accuracy: 0.2517 - loss: 1.3981
Epoch 3/30
21641/21641 ━━━━━━━━━━━━━━━━━━━━ 49s 2ms/step - accuracy: 0.2536 - loss: 1.4062
Epoch 4/30
21641/21641 ━━━━━━━━━━━━━━━━━━━━ 50s 2ms/step - accuracy: 0.2518 - loss: 1.3863
Epoch 5/30
21641/21641 ━━━━━━━━━━━━━━━━━━━━ 49s 2ms/step - accuracy: 0.2516 - loss: 1.4006
Epoch 6/30
21641/21641 ━━━━━━━━━━━━━━━━━━━━ 48s 2ms/step - accuracy: 0.2517 - loss: 1.3915
Epoch 7/30
21641/21641 ━━━━━━━━━━━━━━━━━━━━ 48s 2ms/step - accuracy: 0.2532 - loss: 1.4172
Epoch 8/30
21641/21641 ━━━━━━━━━━━━━━━━━━━━ 82s 2ms/step - accuracy: 0.2526 - loss: 1.3965
Epoch 9/30
21641/21641 ━━━━━━━━━━━━━━━━━━━━ 48s 2ms/step - accuracy: 0.2522 - loss: 1.3990
Epoch 10/30
21641/21641 ━━━━━━━━━━━━━━━━━━━━ 49s 2ms/step - accuracy: 0.2530 - loss: 1.3863
Epoch 11/30
21641/21641 ━━━━━━━━━━━━━━━━━━━━ 48s 2ms/step - accuracy: 0.2515 - loss: 1.

In [ ]:
preds_clases = np.argmax(preds_ts, axis=1)

submission = pd.DataFrame({
    "ID": d_test["ID"],
    "RENDIMIENTO_GLOBAL": preds_clases
})

submission.head()

,ID,RENDIMIENTO_GLOBAL
0,550236,0
1,98545,0
2,499179,0
3,782980,0
4,785185,0


In [ ]:
submission.head(100)
submission["RENDIMIENTO_GLOBAL"].value_counts()

,count
RENDIMIENTO_GLOBAL,
0,296773
3,7
2,3
1,3


In [ ]:
submission["RENDIMIENTO_GLOBAL"] = submission["RENDIMIENTO_GLOBAL"].map({
    0: "bajo",
    1: "medio-bajo",
    2: "medio-alto",
    3: "alto",
}).astype("category")

submission.head()

,ID,RENDIMIENTO_GLOBAL
0,550236,bajo
1,98545,bajo
2,499179,bajo
3,782980,bajo
4,785185,bajo


In [ ]:
submission.to_csv("Andrade1.csv", index=False)

In [ ]:
from google.colab import files
files.download("Andrade1.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>